In [5]:
# simulation of Bernoulli Arm
import random

class BernoulliArm():
  def __init__(self, p):
    self.p = p
  
  def draw(self):
    if random.random() > self.p:
      return 0.0
    else:
      return 1.0


In [6]:
# Convenience functions to return the index of max
def ind_max(x):
  m = max(x)
  return x.index(m)

In [11]:
# Testing Framework
def test_algorithm(algo, arms, num_sims, horizon):
  chosen_arms = [0.0 for i in range(num_sims * horizon)]
  rewards = [0.0 for i in range(num_sims * horizon)]
  cumulative_rewards = [0.0 for i in range(num_sims * horizon)]
  sim_nums = [0.0 for i in range(num_sims * horizon)]
  times = [0.0 for i in range(num_sims * horizon)]
  
  for sim in range(num_sims):
    sim = sim + 1
    algo.initialize(len(arms))
    
    for t in range(horizon):
      t = t + 1
      index = (sim - 1) * horizon + t - 1
      sim_nums[index] = sim
      times[index] = t
      
      chosen_arm = algo.select_arm()
      chosen_arms[index] = chosen_arm
      
      reward = arms[chosen_arms[index]].draw()
      rewards[index] = reward
      
      if t == 1:
        cumulative_rewards[index] = reward
      else:
        cumulative_rewards[index] = cumulative_rewards[index - 1] + reward
      
      algo.update(chosen_arm, reward)
  
  return [sim_nums, times, chosen_arms, rewards, cumulative_rewards]

In [12]:
# Epsilon Greedy Algorithm

import random

def ind_max(x):
  m = max(x)
  return x.index(m)

class EpsilonGreedy():
  def __init__(self, epsilon, counts, values):
    self.epsilon = epsilon
    self.counts = counts
    self.values = values
    return

  def initialize(self, n_arms):
    self.counts = [0 for col in range(n_arms)]
    self.values = [0.0 for col in range(n_arms)]
    return

  def select_arm(self):
    if random.random() > self.epsilon:
      return ind_max(self.values)
    else:
      return random.randrange(len(self.values))
  
  def update(self, chosen_arm, reward):
    self.counts[chosen_arm] = self.counts[chosen_arm] + 1
    n = self.counts[chosen_arm]
    
    value = self.values[chosen_arm]
    new_value = ((n - 1) / float(n)) * value + (1 / float(n)) * reward
    self.values[chosen_arm] = new_value
    return

In [13]:
# UCB1 Algorithm

import math

def ind_max(x):
  m = max(x)
  return x.index(m)

class UCB1():
  def __init__(self, counts, values):
    self.counts = counts
    self.values = values
    return
  
  def initialize(self, n_arms):
    self.counts = [0 for col in range(n_arms)]
    self.values = [0.0 for col in range(n_arms)]
    return
  
  def select_arm(self):
    n_arms = len(self.counts)
    for arm in range(n_arms):
      if self.counts[arm] == 0:
        return arm

    ucb_values = [0.0 for arm in range(n_arms)]
    total_counts = sum(self.counts)
    for arm in range(n_arms):
      bonus = math.sqrt((2 * math.log(total_counts)) / float(self.counts[arm]))
      ucb_values[arm] = self.values[arm] + bonus
    return ind_max(ucb_values)
  
  def update(self, chosen_arm, reward):
    self.counts[chosen_arm] = self.counts[chosen_arm] + 1
    n = self.counts[chosen_arm]

    value = self.values[chosen_arm]
    new_value = ((n - 1) / float(n)) * value + (1 / float(n)) * reward
    self.values[chosen_arm] = new_value
    return


In [18]:
# Thompson Sampling Algorithim

import random as rd

class ThompsonSampling:
    def __init__(self, alpha, beta, succ, fail):
        self.alpha = alpha
        self.beta = beta
        self.succ = succ
        self.fail = fail

    def initialize(self, n_arms):
        self.alpha = [1] * n_arms
        self.beta = [1] * n_arms
        self.succ = [0 for col in range(n_arms)]
        self.fail = [0 for col in range(n_arms)]

    def select_arm(self):
        thetas = list()
        for i in range(len(self.alpha)):
            thetas.append(rd.betavariate(self.succ[i] + self.alpha[i],self.fail[i] + self.beta[i]))
            
                
        return thetas.index(max(thetas))

    def update(self, chosen_arm, reward):
        if reward == 1:
            self.succ[chosen_arm] += 1
        else:
            self.fail[chosen_arm] += 1

In [22]:
# Simulation Settings
import random

random.seed(1)

num_Iteration =1000
num_simulation =1
means = [0.5,0.505 ,0.510 ,0,515 ,0.520]
n_arms = len(means)
arms_ = map((lambda mu: BernoulliArm(mu)), means)
arms = list (arms_)
print("Best arm is " + str(ind_max(means)))

Best arm is 4


In [23]:
# To run a simulation
# Specify the file name to write
# Specify the Algorithm 
# Epsilon loop is only necessary for epsilon Greedy,be careful to indent
import time

#f = open(time.strftime("%Y%m%d-%H%M")+"-eps-" +str(means)+ "- it"+ str(num_Iteration) +"-sim"+ str(num_simulation)+".csv", "w")
#f = open(time.strftime("%Y%m%d-%H%M")+"-ucb-" +str(means)+ "- it"+ str(num_Iteration) +"-sim"+ str(num_simulation)+".csv", "w")
f = open(time.strftime("%Y%m%d-%H%M")+"-bys-" +str(means)+ "- it"+ str(num_Iteration) +"-sim"+ str(num_simulation)+".csv", "w")

#for epsilon in [0.2,0.4,0.6]:
  
  
  #algo = EpsilonGreedy(epsilon, [], [])
  #algo = UCB1([], [])  
algo = ThompsonSampling([], [], [], [])  
algo.initialize(n_arms)
results = test_algorithm(algo, arms, num_simulation, num_Iteration)
for i in range(len(results[0])):
      #f.write(str(epsilon) + ",")
      f.write(",".join([str(results[j][i]) for j in range(len(results))]) + "\n")

f.close()
